# Importo librerías.

In [12]:
#Uso %%capture para ocultar el output. Sacar si hay algun problema en los siguientes
#pasos.
%%capture

!pip install Faker pandas;


# Genero la BD SQLite con datos ficticios.

In [46]:
import sqlite3
from faker import Faker
import random

# Conectar a la base de datos (crea el archivo si no existe)
conn = sqlite3.connect('datos.db')
cursor = conn.cursor()

cursor.executescript('''
DROP TABLE IF EXISTS empleado;
CREATE TABLE empleado (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    pais TEXT NOT NULL,
    nombre TEXT NOT NULL,
    edad INTEGER,
    puesto TEXT,
    activo INTEGER CHECK(activo IN (0, 1)),
    sueldo REAL
);
''')

fake = Faker('es_ES') # Generar datos en español

for _ in range(10000):
    pais = fake.country()
    nombre = fake.name()
    edad = random.randint(22, 65)
    puesto = fake.job()
    activo = random.choice([0, 1])
    sueldo = round(random.uniform(25000.00, 150000.00), 2)

    cursor.execute('''
    INSERT INTO empleado (pais, nombre, edad, puesto, activo, sueldo)
    VALUES (?, ?, ?, ?, ?, ?)
    ''', (pais, nombre, edad, puesto, activo, sueldo))

conn.commit()
conn.close()

## Tablitas de autos.

In [53]:
import sqlite3
from faker import Faker
import random

conn = sqlite3.connect('empleados.db')
cursor = conn.cursor()

cursor.executescript('''
DROP TABLE IF EXISTS modelo_auto;
CREATE TABLE modelo_auto (
    modelo_id INTEGER PRIMARY KEY AUTOINCREMENT,
    nombre_modelo TEXT NOT NULL
);
''')

modelos = ["Sedan", "SUV", "Hatchback", "Pickup", "Coupe"]
for modelo in modelos:
    cursor.execute("INSERT INTO modelo_auto (nombre_modelo) VALUES (?)", (modelo,))

cursor.executescript('''
DROP TABLE IF EXISTS auto_vendido;
CREATE TABLE auto_vendido (
    venta_id INTEGER PRIMARY KEY AUTOINCREMENT,
    modelo_id INTEGER NOT NULL,
    fecha_venta DATE,
    precio REAL,
    FOREIGN KEY (modelo_id) REFERENCES modelo_auto(modelo_id)
);
''')

fake = Faker('es_ES')

for _ in range(1000):
    modelo_id = random.randint(1, len(modelos))
    fecha_venta = fake.date_between(start_date='-2y', end_date='today')
    precio = round(random.uniform(10000.00, 80000.00), 2)

    cursor.execute('''
    INSERT INTO auto_vendido (modelo_id, fecha_venta, precio)
    VALUES (?, ?, ?)
    ''', (modelo_id, fecha_venta, precio))

conn.commit()
conn.close()


# Ejemplos

In [49]:
# @title Consulta Básica {"run":"auto","form-width":"1000px"}
import sqlite3
import pandas as pd

conn = sqlite3.connect('datos.db')

query = "select * from empleado limit 10;" # @param {"type":"string","placeholder":"select * from empleado;"}

df = pd.read_sql_query(query, conn)
conn.close()

df

,id,pais,nombre,edad,puesto,activo,sueldo
0,1,Bahamas,Begoña Bonilla Torrecilla,26,Veterinario,1,59117.11
1,2,Libia,José Ángel Morell Llanos,27,Biólogo,0,37605.20
2,3,Camboya,Mateo Menéndez Galván,35,Administrador de bases de datos,0,138680.86
3,4,Líbano,Julio César Hoz Solana,38,Profesional de medicina alternativa,0,29982.33
4,5,Mozambique,Consuelo Benito Planas,44,Pintor y empapelador,0,99548.50
5,6,Malawi,Fito Hoz Maestre,47,Empleado de ventanillas de informaciones,1,64768.85
6,7,Paraguay,Isidora Ariadna Casal Pavón,28,Director de producción agropecuaria y silvicul...,0,56440.59
7,8,Bosnia y Herzegovina,Rodrigo Abascal Martí,40,Capitán decubierta,1,49290.39
8,9,Camerún,Flavia Tovar Robles,37,Diseñador de productos,0,111452.58
9,10,Bosnia y Herzegovina,Ariel Nogueira Agustí,39,Mecánico y reparador de vehículos de motor,0,110607.91


In [54]:
# @title Consulta con Joins {"run":"none","form-width":"1000px"}
import sqlite3
import pandas as pd

conn = sqlite3.connect('datos.db')

query = "select * from modelo_auto;" # @param {"type":"string","placeholder":"select * from empleado;"}

df = pd.read_sql_query(query, conn)
conn.close()

df

DatabaseError: Execution failed on sql 'select * from modelo_auto;': no such table: modelo_auto

In [38]:
# @title Consulta con Joins {"run":"none","form-width":"1000px"}
import sqlite3
import pandas as pd

conn = sqlite3.connect('empleados.db')

query = "select vendidos.modelo_id, fecha_venta, precio, modelos.nombre_modelo from autos_vendidos vendidos  left join modelos_autos modelos on vendidos.modelo_id = modelos.modelo_id;" # @param {"type":"string","placeholder":"select * from empleados;"}

df = pd.read_sql_query(query, conn)
conn.close()

df

,modelo_id,fecha_venta,precio,nombre_modelo
0,1,2024-02-01,11872.99,Sedan
1,4,2025-03-14,42378.84,Pickup
2,4,2025-05-02,75248.14,Pickup
3,3,2025-05-03,50846.68,Hatchback
4,4,2025-03-05,56257.31,Pickup
...,...,...,...,...
995,4,2025-03-11,78070.09,Pickup
996,4,2024-07-23,12840.12,Pickup
997,3,2023-11-11,43577.65,Hatchback
998,5,2025-04-24,24753.36,Coupe


In [ ]:
# @title Consulta con Algo Mas {"run":"auto","form-width":"1000px"}
import sqlite3
import pandas as pd

conn = sqlite3.connect('empleados.db')

query = "select nombre,edad,sueldo from empleados where activo = 1 and pais = 'Argentina' order by edad desc;" # @param {"type":"string","placeholder":"select * from empleados;"}

df = pd.read_sql_query(query, conn)
conn.close()

df

In [ ]:
# @title Análisis - Top 10 Plantilla mas Veterana {"run":"auto","form-width":"1000px"}
import sqlite3
import pandas as pd

conn = sqlite3.connect('empleados.db')

query = "select pais, count(*) q_empleados, round(avg(edad),2) prom_edad from empleados group by pais order by avg(edad) desc limit 10; " # @param {"type":"string","placeholder":"select * from empleados;"}

df = pd.read_sql_query(query, conn)
conn.close()

df

In [ ]:
# @title Análisis - Top 3 Sueldos Por Pais {"run":"auto","form-width":"1000px"}
import sqlite3
import pandas as pd

conn = sqlite3.connect('empleados.db')

query = "select * from (select pais, id,nombre, sueldo, rank() over (partition by pais order by sueldo desc) ranking from empleados where activo = 1) where ranking < 4 order by pais" # @param {"type":"string","placeholder":"select * from empleados;"}

df = pd.read_sql_query(query, conn)
conn.close()

df

,pais,id,nombre,sueldo,ranking
0,Afganistán,7959,Ruperta Alcántara Briones,142982.68,1
1,Afganistán,8700,Gala Arias Mulet,139029.26,2
2,Afganistán,4617,Nacho Goicoechea Tejera,138119.38,3
3,Albania,612,Casandra Serna Plana,144920.41,1
4,Albania,2589,Cristóbal José Ángel Bernat Valenzuela,140492.62,2
...,...,...,...,...,...
574,Zambia,72,Nayara Montenegro Batlle,148835.77,2
575,Zambia,7456,Marianela Eusebia Lara Cantón,146985.45,3
576,Zimbabwe,1004,Aránzazu Nebot Salvà,149267.51,1
577,Zimbabwe,4934,Arcelia Girona Lozano,145984.55,2
